Output: flight_times.csv (europe_flights.csv)

In [2]:
# set up
import requests
import pandas as pd
import random

airlab_api_params = {
    "api_key" : "8ffb5e97-7a50-4773-b2d9-8638ec08bc75",
}

airlab_ex_param = {
    "api_key" : "8ffb5e97-7a50-4773-b2d9-8638ec08bc75",
    "dep_iata" : "ANR"
}

AirLabs API 

to get information about times schedules in Europe we draw info from airlbas api. In our project we subset our citie

In [3]:
PATH_IATA_CODES = "./datasets/airport-codes.csv" # we get largest largest airports all over the world

Summary and filtering largest airports

In [6]:
EURO_IATA_CODES = pd.read_csv(PATH_IATA_CODES)
EURO_IATA_CODES['type'].unique()

array(['heliport', 'small_airport', 'closed', 'seaplane_base',
       'balloonport', 'medium_airport', 'large_airport'], dtype=object)

In [9]:
# EURO_IATA_CODES.loc[EURO_IATA_CODES["type"] == 'large_airport' & EURO_IATA_CODES['continent'] == 'EU']
EU_LARGE = EURO_IATA_CODES.loc[(EURO_IATA_CODES["type"].isin(['large_airport','medium_airport']))  &  (EURO_IATA_CODES['continent'].isin(["EU"]) )][["name","iata_code","iso_region","iso_country","coordinates"]]

## exclude islands and unaccessible airports

airports_to_exclude= ["HER", "NaN","PMI", "LPA", "TFS", "TFN"]
# island countries
countries_to_exclude = ["IS","IE"]

FINAL_AIRPORTS = EU_LARGE.loc[(~EU_LARGE['iso_country'].isin(countries_to_exclude))  
                              & (~EU_LARGE['iata_code'].isin(airports_to_exclude))
                              & (~EU_LARGE['iata_code'].isna())].reset_index(drop=True)
FINAL_AIRPORTS


,name,iata_code,iso_region,iso_country,coordinates
0,Bezmer Air Base,JAM,BG-28,BG,"26.3521995544, 42.4548988342"
1,PriÅ¡tina International Airport,PRN,XK-01,XK,"21.035801, 42.5728"
2,Antwerp International Airport (Deurne),ANR,BE-VAN,BE,"4.46027994156, 51.1893997192"
3,Brussels Airport,BRU,BE-BRU,BE,"4.48443984985, 50.901401519800004"
4,Brussels South Charleroi Airport,CRL,BE-WHT,BE,"4.45382, 50.459202"
...,...,...,...,...,...
783,Balakovo Airport,BWO,RU-SAR,RU,"47.7456016541, 51.8582992554"
784,Saratov Central Airport,RTW,RU-SAR,RU,"46.04669952392578, 51.564998626708984"
785,Ufa International Airport,UFA,RU-BA,RU,"55.874401092529, 54.557498931885"
786,Kurumoch International Airport,KUF,RU-SAM,RU,"50.16429901123, 53.504901885986"


Save selected large airports to CSV

In [10]:
FINAL_AIRPORTS.to_csv("./datasets/AIRPORT_SET.csv")

In [64]:
FINAL_AIRPORTS.loc[FINAL_AIRPORTS['iso_country'] == "SK"]

,name,iata_code,iso_region,iso_country,coordinates
663,M. R. Å tefÃ¡nik Airport,BTS,SK-BL,SK,"17.21269989013672, 48.17020034790039"
664,KoÅ¡ice Airport,KSC,SK-KI,SK,"21.241100311279297, 48.66310119628906"
665,PieÅ¡Å¥any Airport,PZY,SK-TA,SK,"17.828399658203125, 48.62519836425781"
666,SliaÄ Airport,SLD,SK-BC,SK,"19.13409996032715, 48.63779830932617"
667,Poprad-Tatry Airport,TAT,SK-KI,SK,"20.2411003113, 49.073600769"
668,Å½ilina Airport,ILZ,SK-ZI,SK,"18.6135005951, 49.231498718299996"


In [36]:
CODES = list(FINAL_AIRPORTS["iata_code"])

sample_CODES = random.sample(CODES, 50)
sample_CODES

['ZIA',
 'EDI',
 'SIR',
 'LBC',
 'TIV',
 'MRS',
 'DGP',
 'ZMG',
 'SKG',
 'SZW',
 'CMF',
 'QLT',
 'CSF',
 'MIK',
 'GOA',
 'MOL',
 'LWR',
 'RYG',
 'BLK',
 'MST',
 'NTE',
 'ZTH',
 'GLZ',
 'LHA',
 'LLA',
 'HVG',
 'AOK',
 'STN',
 'HZB',
 'RRS',
 'BZY',
 'PEV',
 'AOI',
 'EXT',
 'BVG',
 'CQF',
 'SVO',
 'JKG',
 'BJF',
 'EIB',
 'VRK',
 'BAY',
 'EPU',
 'SJJ',
 'NHT',
 'GEV',
 'SEK',
 'BWF',
 'LME',
 'FCO']

In [28]:
"CDG" in sample_CODES

False

In [84]:
def get_flight_duration(codes):
    
    # download
    data = []
    for code in codes:
        queryX = {
            "api_key" : "8ffb5e97-7a50-4773-b2d9-8638ec08bc75",
            "dep_iata" :  f"{code}"
        }

        req_get = requests.get("https://airlabs.co/api/v9/schedules?",params= queryX)        
        DF = parse_response(req_get)
        if len(DF) > 0:
            DF = filter_clean_codes(DF, codes)
            data.append(DF)

    DF["route"] = DF.apply(lambda row: set([row["dep"], row["arr"]]), axis=1)
    DF = DF.loc[["route","time"]]
    DF.drop_duplicates(inplace=True, ignore_index=True)
  
    # concat all and return 
    return pd.concat(data, ignore_index=True)



def parse_response(api_response):
    """
    Keyword arguments:
    api_response -- GET response
    Return: pd.DataFrame of flight routes
    """

    j_resp = api_response.json()
    flight_list = j_resp['response']
    
    flights = []
    for fl in flight_list:
        one_flight = {"dep":fl['dep_iata'],
                      "arr":fl['arr_iata'],
                      "time":fl['duration']}
        flights.append(one_flight)

    print(flight_list)
    return pd.DataFrame(flights)

def filter_clean_codes(flight_set, codes):
    df = flight_set[ (flight_set["dep"].isin(codes)) & (flight_set["arr"].isin(codes))]
    # uDF = df.loc[["route","time"]]
    return df.drop_duplicates(ignore_index=True)

In [85]:
Flights= get_flight_duration(CODES)
Flights.head()

KeyError: 'response'

In [68]:
ksc = Flights.loc[ (Flights["dep"]== "KSC") | (Flights["arr"]== "KSC")].reset_index(drop=True)

In [88]:
Flights.to_csv("./datasets/Flights1.csv")

In [96]:
Flights["route"] = Flights.apply(lambda row: str( set([row["dep"], row["arr"]]) ), axis=1)
Flights


,dep,arr,time,route
0,PRN,STR,125,"{'PRN', 'STR'}"
1,PRN,DUS,160,"{'DUS', 'PRN'}"
2,PRN,DUS,165,"{'DUS', 'PRN'}"
3,PRN,MUC,120,"{'MUC', 'PRN'}"
4,PRN,HAM,165,"{'PRN', 'HAM'}"
...,...,...,...,...
2271,KUF,LED,145,"{'KUF', 'LED'}"
2272,KUF,VKO,120,"{'VKO', 'KUF'}"
2273,KUF,LED,150,"{'KUF', 'LED'}"
2274,KUF,SVO,115,"{'SVO', 'KUF'}"


In [107]:
FlightsShort = Flights[["route", "time"]].groupby(["route"]).mean()
FlightsShort.reset_index(inplace=True)
FlightsShort["route"] = FlightsShort["route"].apply(lambda x: eval(x))
FlightsShort["dep"] 

,route,time
0,"{AAL, AMS}",85.0
1,"{AAQ, LED}",170.0
2,"{AMS, ABZ}",85.0
3,"{ABZ, LGW}",95.0
4,"{ACE, AGP}",130.0
...,...,...
1464,"{ZRH, OTP}",150.0
1465,"{ZRH, PRN}",127.5
1466,"{ZRH, RVN}",205.0
1467,"{ZRH, STR}",45.0


In [109]:
FlightsShort.to_csv("./datasets/europe_flights.csv")

In [17]:
queryCDG = {
    "api_key" : "8ffb5e97-7a50-4773-b2d9-8638ec08bc75",
    "dep_iata" :  "CDG"   
}


res_ex = requests.get("https://airlabs.co/api/v9/schedules?",params= queryCDG)
js_res = res_ex.json()
js_res["response"]

[{'airline_iata': 'AH',
  'airline_icao': 'DAH',
  'flight_iata': 'AH1149',
  'flight_icao': 'DAH1149',
  'flight_number': '1149',
  'dep_iata': 'CDG',
  'dep_icao': 'LFPG',
  'dep_terminal': '2D',
  'dep_gate': 'B31',
  'dep_time': '2024-01-20 14:00',
  'dep_time_utc': '2024-01-20 13:00',
  'dep_estimated': '2024-01-20 14:34',
  'dep_estimated_utc': '2024-01-20 13:34',
  'dep_actual': '2024-01-20 14:34',
  'dep_actual_utc': '2024-01-20 13:34',
  'arr_iata': 'CFK',
  'arr_icao': 'DAOI',
  'arr_terminal': None,
  'arr_gate': None,
  'arr_baggage': None,
  'arr_time': '2024-01-20 16:35',
  'arr_time_utc': '2024-01-20 15:35',
  'arr_estimated': '2024-01-20 16:43',
  'arr_estimated_utc': '2024-01-20 15:43',
  'cs_airline_iata': None,
  'cs_flight_number': None,
  'cs_flight_iata': None,
  'status': 'active',
  'duration': 155,
  'delayed': 8,
  'dep_delayed': 34,
  'arr_delayed': 8,
  'aircraft_icao': 'B738',
  'arr_time_ts': 1705764900,
  'dep_time_ts': 1705755600,
  'arr_estimated_ts': 1

Get sample of api request for connections between given airports 


In [43]:
DF = parse_response(res_ex)
DF.head()

,dep,arr,time
0,CDG,CFK,155
1,CDG,ALG,135
2,CDG,FRA,80
3,CDG,ATL,590
4,CDG,ATL,590


In [53]:
filter_clean_codes(DF, CODES)

,dep,arr,time
0,CDG,FRA,80
1,CDG,AMS,80
2,CDG,OSL,140
3,CDG,GOT,125
4,CDG,CPH,115
5,CDG,TLS,80
6,CDG,ZRH,75
7,CDG,PRG,105
8,CDG,ARN,160


## Measure performance / Analytics of Flights

In [115]:
Flights_europe = pd.read_csv("./datasets/europe_flights.csv")

ValueError: The value of index_col couldn't be 'True'

In [116]:
Flights_europe

,Unnamed: 0,route,time
0,0,"{'AAL', 'AMS'}",85.0
1,1,"{'AAQ', 'LED'}",170.0
2,2,"{'AMS', 'ABZ'}",85.0
3,3,"{'ABZ', 'LGW'}",95.0
4,4,"{'ACE', 'AGP'}",130.0
...,...,...,...
1464,1464,"{'ZRH', 'OTP'}",150.0
1465,1465,"{'ZRH', 'PRN'}",127.5
1466,1466,"{'ZRH', 'RVN'}",205.0
1467,1467,"{'ZRH', 'STR'}",45.0
